**Задание**

    продумать написать тему курсового проекта
    подготовить данные для обучения
    продумать интенты на которые вы будете обучаться

**Для сдачи задания необходимо написать**

    на какую тему ваш бот будет общаться (на каких датасетах планируете обучать модели)
    написать на какие интенты вы будете обучать бота


# Решение

**Lord of the rings ChatBot**

`Задача чат-бота - общаться с людьми в тематике Властелина колец. Также чат-бот может сыграть в игру типо простого "Акинатора" - угадывать персонажа, которого загадал пользователь. Помимо этого бот может придумывать (генерировать) имена в стилистике Властелина колец.`

*Интенты:*
1. Болталка: чат-бот ведет диалог в стилистике Властелина колец
2. Игра "Угадай персонажа"
3. Генератор *имен*



In [15]:
# Загружаем библиотеки
! pip install pymorphy2
! pip install annoy
! pip install stop_words

import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import string

import io
import chardet
import os
import codecs

import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
import annoy
import gensim.models

from stop_words import get_stop_words
import nltk
import nltk.data
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

!pip install wikipedia

import time
import wikipedia
wikipedia.set_lang("ru")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=4c4acc70f978c1c0a5c948dcf28ae57320c101c0b3ee5f9fc5fad45b87fd39a0
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


**Болталка: чат-бот ведет диалог в стилистике Властелина колец**

Для первой задачи - обучим нейросеть на тексте Властелина колец. Пользователь пишет сообщение в чат-бот, модель ищет максимально похожий текст в столбце "question" и выдает ответ с минимальной дистанцией. Если похожего ответа нет, чат бот выдает ответ из вики

In [16]:
# Загружаем данные и сразу преобразуем их в токены
tokenizer1 = nltk.data.load('tokenizers/punkt/russian.pickle')
data = open("/content/lotr.txt").read()


# Предложения вопросы
tokens_q = tokenizer1.tokenize(data)
print(len(tokens_q)), print(tokens_q[:3])

5204
['Мир изменился.', 'Я чувствую это в воде, чувствую в земле, ощущаю в воздухе.', 'Многое из того, что было - ушло.']


(None, None)

In [17]:
# Предложения ответы: из получившегося списка вопросов
# формируем ответы - каждое последующее предложение является ответом для предыдущего

tokens_a = tokens_q[1:]
tokens_a.append(tokens_q[:1])
print(len(tokens_a)), print(tokens_a[:3])

5204
['Я чувствую это в воде, чувствую в земле, ощущаю в воздухе.', 'Многое из того, что было - ушло.', 'И не осталось тех, кто помнит об этом.']


(None, None)

In [18]:
df = pd.DataFrame({'question': tokens_q, 'answer': tokens_a})
df.sample(2)

,question,answer
1083,Что это?,Ничего.
3015,Он должен уничтожить его!,Поймите же куда мы идём.


Функции для модели

In [19]:
# Преобразование данных

morpher = MorphAnalyzer()
sw = get_stop_words("ru") + stopwords.words('russian')
punkt = list(string.punctuation)
noise = set(sw + punkt)

def preprocess_txt(text):
    return [t for t in [morpher.parse(token)[0].normal_form 
                                for token in word_tokenize(text)] if t not in noise]


def start_prep(data):
    result = []

    for text in tqdm(data):
      prep = preprocess_txt(text)
      result.append(prep)

    return result

In [20]:
# Процедура поиска ближайших вопросов
def get_model_index(model):
    model_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

    index_map = {}
    counter = 0

    for question in df.question:
        n_model = 0
        index_map[counter] = question
        que_prep = preprocess_txt(question)
        
        vector_model = np.zeros(300) #вектор твита

        for word in que_prep:
            if word in model:
                vector_model += model[word]
                n_model += 1

        if n_model > 0:
            vector_model = vector_model / n_model #нормализуем вектор=
       
        model_index.add_item(counter, vector_model) #добавляем элемент для индексации
     
        counter += 1

        if counter > 100000:
            break

    model_index.build(100) #из индексов создаем кластеры
    return model_index, index_map

In [21]:
# Функция поиска ближайших соседей
def get_response(que, index, model, index_map, df):
    token = preprocess_txt(que)
    vector = np.zeros(300)
    norm = 0

    for word in token:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    similar_que = index.get_nns_by_vector(vector, 15, include_distances=True)
    
    result = pd.DataFrame({'question': [index_map[i]  for i in similar_que[0]], 
                           'distance' : similar_que[1], 'answer': [df[df.index == i].answer.values[0] for i in similar_que[0]]})
    

    return result

In [22]:
tokens = start_prep(df.question)
modelFT = FastText(sentences=tokens, size=300, min_count=1, window=5, workers=8)
ft_index, index_map = get_model_index(modelFT.wv)

100%|██████████| 5204/5204 [00:07<00:00, 742.13it/s]


Пример работы модели

In [23]:
smp = 'Арагорн?'

print(smp)
# get_response(smp, ft_index, modelFT.wv, index_map, df)

# single answer
rand = random.randint(0,14)
resp = get_response(smp, ft_index, modelFT.wv, index_map, df)
print(resp.answer.values[rand])

Арагорн?
У них не было чести при жизни, нету и после смерти.


In [24]:
smp = 'Расскажи о средиземье'

print(smp)
# get_response(smp, ft_index, modelFT.wv, index_map, df)

# single answer
rand = random.randint(0,14)
resp = get_response(smp, ft_index, modelFT.wv, index_map, df)
print(resp.answer.values[rand])

Расскажи о средиземье
Ужасная искалеченная форма жизни.


In [25]:
resp

,question,distance,answer
0,"Расскажи мне всё, всё что знаешь.",0.027144,Вот как ты послужил своему городу.
1,В Средиземье кое-что начинает гнить.,0.028718,"То, чего ты не увидел."
2,В наших силах решать только что делать со врем...,0.030493,"В мире действуют иные силы, Фродо, кроме воли ..."
3,"Кажется, приспособился.",0.030620,Арагорн!
4,Развяжи его и он зарежет нас спящими!,0.030629,Ты знаешь путь в Мордор?
5,"Он собирает у себя всё зло, и скоро он соберёт...",0.030809,Ты знаешь это?
6,Ты позвал нас биться.,0.030975,Здесь не место хоббиту!
7,Она видела как её мать умерла от горя и остала...,0.031024,"Гимли...\nНет, я не могу."
8,"Как думаешь, стоит поделиться с Древнем?",0.031237,Поделиться?
9,Хозяин просил показать ему путь в Мордор.,0.031522,"Добрый Смегол показал, как хозяин просил."


**Игра "Угадай персонажа"**

Пользователь вводит описание персонажа, модель делает предсказание, к кому из персонажей это описание может относиться.

In [111]:
#В качестве данных возьмем wiki описание персонажей Властелина колец
heroes = ['Арагорн', 'Арвен', 'Бильбо Бэггинс', 'Том Бомбадил', 'Боромир', 'Фродо Бэггинс', 'Галадриэль', 
          'Гимли', 'Голлум', 'Грима Червеуст', 'Гэндальф', 'Дэнетор II', 'Исильдур', 'Келеборн', 'Кирдан Корабел', 
          'Король-чародей Ангмара', 'Леголас', 'Мериадок Брендибак', 'Перегрин Тук', 'Радагаст', 'Саруман', 'Саурон', 'Сэмуайз Гэмджи', 
          'Теоден', 'Трандуил', 'Фарамир', 'Элронд', 'Эовин', 'Эомер']

heroes_data = {'description': [], 
               'character': []}

for hero in tqdm(heroes):
  wiki = wikipedia.search(hero)
  ans = wikipedia.page(wiki[0]).content
  heroes_data['description'].append(ans)
  heroes_data['character'].append(hero)

100%|██████████| 29/29 [00:28<00:00,  1.02it/s]


In [308]:
df2 = pd.DataFrame.from_dict(heroes_data)
df2.head(2)

,description,character
0,А́рагорн — один из главных персонажей романа «...,Арагорн
1,А́рвен Ундо́миэль (синд. Arwen Undómiel) — пер...,Арвен


In [113]:
# Импорт дополнительных библиотек
from sklearn.model_selection import train_test_split
from string import punctuation
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping 

In [309]:
import re

# Преобразование данных
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
cats = set(df2.character.str.lower().to_list()) # исключаем из обучающих текстов имена самих персонажей
morpher = MorphAnalyzer()

def preprocess_text(txt):
    #print('raw: ', txt)
    txt = str(txt)
    #print('to str: ', txt)
    txt = txt.lower()
    #print('lower: ', txt)
    txt = ''.join([i for i in txt if not i.isdigit()])
    #print('no digits: ', txt)
    #txt = ''.join([i for i in txt if i.isalpha()])
    txt = re.sub('[()!@#$"«»/]', '', txt)
    #print('no sym: ', txt)
    txt = "".join(c for c in txt if c not in exclude)
    #print('exclude: ', txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split(' ') if word not in sw]
    #print('morpher: ', txt)
    txt = " ".join(c for c in txt if c not in cats)
    #print('wo cats: ', txt)
    return "".join(txt)

In [310]:
tqdm.pandas()
df2['description'] = df2['description'].progress_apply(preprocess_text)

100%|██████████| 29/29 [00:08<00:00,  3.57it/s]


In [311]:
df2['description'].sample(2)

22    сэ́муайза гэ́мджи англ samwise gamgee вариант ...
5     фро́дый бэ́ггинс англ frodo baggins вестрон ma...
Name: description, dtype: object

In [312]:
from nltk.tokenize import word_tokenize
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')


# Проведем токенизацию по 3-ем словам
upd_df2 = {'description': [],
           'character': []}

for char in df2.character.unique():
  data = df2[df2.character == char].description.to_list()
  for d in data:
    desc = tokenizer.tokenize(d)
    for el in desc:
      #print(el)
      el = el.replace('—', ' ')
      el = re.sub('[a-zA-Z]', '', el)
      words = el.split(' ')
      words = [word for word in words if word != '']
      words = [word for word in words if len(word) >= 3]
      #print(words)
      n_min = 0
      n_max = len(words)
      
      while n_min != n_max-2:
        tok = [''.join(word) for word in words[n_min:n_min+3]]
        n_min += 1
        #print(tok)

        upd_df2['description'].append(preprocess_text(tok))
        upd_df2['character'].append(char)

In [313]:
df2 = pd.DataFrame.from_dict(upd_df2)
df2.sample(2)

,description,character
14315,король элессара советник,Перегрин Тук
21982,предупредить хоббит древний,Фарамир


In [317]:
df2.shape

(24081, 2)

In [318]:
df2 = df2.drop_duplicates(subset=['description'], keep='first')

In [320]:
df2.shape

(22997, 2)

In [321]:
#Разбиваем на трейн и тест
df_train, df_test = train_test_split(df2, test_size=0.10, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((20697, 2), (2300, 2))

In [322]:
df_train['description'].sample(5)

11919             собиратель мох ссылаться
5286         магический камень одноимённый
1179              верёвка привязывать нога
19228             войти диссонанс мелькора
12361    штаны словосочетание использовать
Name: description, dtype: object

In [323]:
text_corpus_train = df_train['description'].values
text_corpus_test = df_test['description'].values

In [324]:
from keras.preprocessing.text import Tokenizer

# Определяем токенайзер
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')

# Обучаем токенайзер на тренировочном наборе данных - > получаем готовый словарь
tokenizer.fit_on_texts(text_corpus_train)

In [325]:
# На основе полученного словаря токены переводятся в их id и сформировываем последовательности sequences

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

In [326]:
# всего слов в словаре
word_count = len(tokenizer.index_word) + 1

# находим максимальную длину будущей последовательности в тренировочном наборе
training_length = max([len(i.split()) for i in text_corpus_train])

word_count, training_length

(6263, 3)

In [327]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

X_train[5]

array([ 130, 2660,   47], dtype=int32)

In [328]:
# Преобразование значений y
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['character']) 
test_enc_labels = le.transform(df_test['character'])
le.classes_

array(['Арагорн', 'Арвен', 'Бильбо Бэггинс', 'Боромир', 'Галадриэль',
       'Гимли', 'Голлум', 'Грима Червеуст', 'Гэндальф', 'Дэнетор II',
       'Исильдур', 'Келеборн', 'Кирдан Корабел', 'Король-чародей Ангмара',
       'Леголас', 'Мериадок Брендибак', 'Перегрин Тук', 'Радагаст',
       'Саруман', 'Саурон', 'Сэмуайз Гэмджи', 'Теоден', 'Том Бомбадил',
       'Трандуил', 'Фарамир', 'Фродо Бэггинс', 'Элронд', 'Эовин', 'Эомер'],
      dtype=object)

In [329]:
num_classes = df_train['character'].nunique()
y_train = to_categorical(train_enc_labels, num_classes)
y_test = to_categorical(test_enc_labels, num_classes)

In [330]:
y_train.shape, y_train[3]

((20697, 29),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [332]:
%%time
early_stopping=EarlyStopping(monitor='val_loss', patience = 1, mode = 'min') 

# RNN - > CNN
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=num_classes, 
              trainable=True,
              mask_zero=True))

model.add(Masking(mask_value=0.0))

model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(64, 3, padding = 'same', activation = 'relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) 

model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=25,
                    epochs=40,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/40
828/828 [==============================] - 24s 24ms/step - loss: 0.2071 - accuracy: 0.0814 - val_loss: 0.1422 - val_accuracy: 0.1322
Epoch 2/40
828/828 [==============================] - 16s 20ms/step - loss: 0.1578 - accuracy: 0.1285 - val_loss: 0.1407 - val_accuracy: 0.1322
Epoch 3/40
828/828 [==============================] - 17s 20ms/step - loss: 0.1538 - accuracy: 0.1328 - val_loss: 0.1395 - val_accuracy: 0.1322
Epoch 4/40
828/828 [==============================] - 17s 20ms/step - loss: 0.1466 - accuracy: 0.1385 - val_loss: 0.1292 - val_accuracy: 0.1530
Epoch 5/40
828/828 [==============================] - 18s 21ms/step - loss: 0.1376 - accuracy: 0.1549 - val_loss: 0.1258 - val_accuracy: 0.1517
Epoch 6/40
828/828 [==============================] - 16s 20ms/step - loss: 0.1325 - accuracy: 0.1599 - val_loss: 0.1238 - val_accuracy: 0.1604
Epoch 7/40
828/828 [==============================] - 17s 20ms/step - loss: 0.1288 - accuracy: 0.1666 - val_loss: 0.1220 - val_accuracy:

In [333]:
predictions = model.predict(X_test)

In [334]:
preds = []
for pred in predictions:
    preds.append(pred.argmax())
    
len(preds), preds[:3]

(2300, [25, 7, 22])

In [335]:
dec_preds = le.inverse_transform(preds)
dec_preds[:3]

array(['Фродо Бэггинс', 'Грима Червеуст', 'Том Бомбадил'], dtype=object)

In [336]:
cols = ['description', 'character']
vals = df_test[cols]
vals['preds'] = dec_preds
vals.sample(5)

,description,character,preds
1917,андуин сарумана правда,Радагаст,Боромир
2064,упасть колено корольчародый,Король-чародей Ангмара,Король-чародей Ангмара
2057,начало сделать начать,Саурон,Саурон
255,рост сила дол,Саурон,Гэндальф
549,тайно вернуться мордорnnn,Саурон,Саурон


Пример работы модели

In [337]:
def prep_input_text(text):
  text = preprocess_text(text)
  text_list = []
  text_list.append(text)
  sequences = tokenizer.texts_to_sequences(text_list)
  pad = pad_sequences(sequences, maxlen=training_length)
  return pad
  
def make_pred(pad):
  predictions_single = model.predict(pad)
  preds = [predictions_single.argmax()]
  return le.inverse_transform(preds)

In [344]:
inp = 'волшебник'
print('Загадайте героя: ', inp)

pred = make_pred(prep_input_text(inp))
print('Ответ: ', pred)

Загадайте героя:  волшебник
Ответ:  ['Гэндальф']


In [339]:
inp = 'моя прелесть'
print('Загадайте героя: ', inp)

pred = make_pred(prep_input_text(inp))
print('Ответ: ', pred)

Загадайте героя:  моя прелесть
Ответ:  ['Голлум']


In [349]:
inp = 'уничтожил кольцо'
print('Загадайте героя: ', inp)

pred = make_pred(prep_input_text(inp))
print('Ответ: ', pred)

Загадайте героя:  уничтожил кольцо
Ответ:  ['Фродо Бэггинс']


**Генератор имен**

Пользователю предлагается задать начальную букву имени, бот в ответ выдаст сгенерированное имя, похожее на эльфийское. Для этой задачи напишем простой алгоритм, который будет основываться на списке имен из Властелина колец и генерировать случайное.

In [54]:
# Загружаем данные
df3 = pd.read_excel('lotr_names.xlsx')
df3.sample(3)

,name
4,Селебриан
55,Древобрад
19,Фарамир


In [55]:
# Разобъем имена на фонемы и создадим словарь по первым буквам

fons = {'а': [], 'б': [], 'в': [], 'г': [], 'д': [], 'е': [], 'ё': [], 'ж': [], 'з': [], 'и': [], 'й': [], 'к': [], 'л': [], 'м': [], 'н': [],
        'о': [], 'п': [], 'р': [], 'с': [], 'т': [], 'у': [], 'ф': [], 'х': [], 'ц': [], 'ч': [], 'ш': [], 'щ': [], 'ы':[], 'ъ':[], 'ь': [], 'э': [], 'ю': [], 'я': [],
        '-': []}

for name in df3.name.str.lower().to_list():
  #print(name, len(name))
  n_min = 0
  n_max = len(name)
  while n_min != n_max:
    fon = name[n_min:n_min+3]
    fl = fon[:1]
    fons[fl].append(fon)
    n_min +=1


In [56]:
def gen_name(letter):
  letter = letter.lower()
  vars = fons[letter]
  r = random.randint(0, len(vars))
  name = vars[r]
  name_length = random.randint(1, 3)
  
  while name_length:
    
    try:
      nl = random.choice(list(fons.keys()))[0]
      var_nl = fons[nl]
      r_nl = random.randint(0, len(var_nl))
      name += var_nl[r_nl]
    except IndexError:
      pass

    name_length -= 1

  return name.title()

In [57]:
let = 'о'
print('Ваше имя: ', gen_name(let))

Ваше имя:  Орльд


**Пишем чат бота**

In [58]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 29.2 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.6.1-py3-none-any.whl size=191363 sha256=b40e47a21f59d5b5d8960ef8bb2e190690995d0369175fbd7e1e2315bd003b9e
  Stored in directory: /root/.cache/pip/wheels/b0/df/78/4b57335f539e922a1949f23f5c8991b8defafa462d9e74913f
Successfully built pyTelegramBotAPI


In [59]:
import telebot
from telebot import types

In [60]:
token='5493296466:AAHzZ3E94qK6OZ8jPu4EA9Geu7x_R4AnMFs'

In [61]:
#Базовые сообщения бота

welcome_text = 'Приветствую тебя в Средиземье, друг! '\
                'Мы можем просто поболтать о разном, поиграть в игру "Угадай персонажа" '\
                'или же я могу придумать тебе новое имя, которому позавидует сам Толкиен. Чем бы ты хотел заняться? '\

back_text = 'Я умею много чего еще. Можем просто поболтать о разном, поиграть в игру "Угадай персонажа",'\
               'или же я могу придумать тебе новое имя, которому позавидует сам Толкиен. Чем бы ты хотел заняться? '

reply_auto = 'Хорошо, дай мне минутку!'
no_reply = 'Извини, кажется я не понял, что ты хочешь. Может попробуем еще раз?'
welcome_start = 'Привет-привет, так чем ты хочешь заняться? Я могу просто болтать, играть в угадайку и придумывать имена.'
hay_start = 'У меня отлично! Хочешь просто поболтать? Давай представим, что мы в Средиземье - начинай рассказ, я подхвачу'
talk_start = 'Хорошо, о чем ты хочешь поболтать?'
play_start = 'Отлично, давай сыграем! Загадай героя и напиши мне какой-нибудь факт о нем.'
game_start = 'Не вопрос, сейчас придумаем тебе имя! Напиши первую букву будущего имени.'

welcome_tok = ['приве', 'здравств', 'расти', 'хай', 'hello', 'hi']
talk_tok = ['говор', 'болта', 'расска', 'скаж', 'чат']
game_tok = ['игр', 'угада', 'весел']
names_tok = ['имя', 'имена', 'назов', 'назва']
hay_tok = ['дел', 'как ты', 'как дела', 'как оно', 'как настроение', 'что нового']

In [358]:
# Функция получения результата модели 
# Если ответ модели плох - заменяем его ответом из википедии

def get_phrase(text):
  resp = get_response(text, ft_index, modelFT.wv, index_map, df)
  model_res = resp.answer.values[0]
  distance_res = resp[resp.answer == model_res].distance.values
  if distance_res == 0.0:
    return model_res
  else:
    wiki_s = wikipedia.search(text)
    try:
      ans = wikipedia.page(wiki_s[0]).content
      wiki_res = tokenizer1.tokenize(ans)[:7]
      return wiki_res
    
    except Exception as e:
      return 'Кажется, я не понимаю, о чем ты хочешь поболтать'


In [359]:
get_phrase('Кто ты, странник?')

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


'Кажется, я не понимаю, о чем ты хочешь поболтать'

In [63]:
# Функция получения результата модели 
def get_answer(text):
  return make_pred(prep_input_text(text))

In [64]:
get_answer('носитель кольца')

array(['Том Бомбадил'], dtype=object)

In [65]:
# Функция получения результата имени
def get_name(text):
  return gen_name(text)

In [ ]:
bot=telebot.TeleBot(token)

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, welcome_text)

@bot.message_handler(commands=['back'])
def back_message(message):
    bot.send_message(message.chat.id, back_text)

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    if True in [tok in message.text.lower() for tok in talk_tok]:
        bot.send_message(message.from_user.id, talk_start)
        bot.register_next_step_handler(message, talk)
        
    elif True in [tok in message.text.lower() for tok in game_tok]:
        bot.send_message(message.from_user.id, play_start)
        bot.register_next_step_handler(message, play)
        
    elif True in [tok in message.text.lower() for tok in names_tok]:
        bot.send_message(message.from_user.id, game_start)
        bot.register_next_step_handler(message, names)
        
    elif True in [tok in message.text.lower() for tok in welcome_tok]:
        bot.send_message(message.from_user.id, welcome_start)  
            
    elif True in [tok in message.text.lower() for tok in hay_tok]:
        bot.send_message(message.from_user.id, hay_start)
        bot.register_next_step_handler(message, talk)
        
    else:
        bot.send_message(message.from_user.id, no_reply)
              
def talk(message):
    inp = message.text
    bot.send_message(message.from_user.id, get_phrase(inp))
    time.sleep(2)
    bot.send_message(message.from_user.id, 'Хочешь продолжить? (да/нет)')
    bot.register_next_step_handler(message, choise_talk)
    
def play(message):
    inp = message.text
    bot.send_message(message.from_user.id, get_answer(inp))
    time.sleep(2)
    bot.send_message(message.from_user.id, 'Хочешь продолжить? (да/нет)')
    bot.register_next_step_handler(message, choise_play)

def names(message):
    inp = message.text
    bot.send_message(message.from_user.id, get_name(inp))
    time.sleep(2)
    bot.send_message(message.from_user.id, 'Хочешь продолжить? (да/нет)')
    bot.register_next_step_handler(message, choise_names)

def choise_talk(message):
    inp = message.text
    if inp.lower() == 'да':
      bot.send_message(message.from_user.id, talk_start)
      bot.register_next_step_handler(message, talk)
    else:
      bot.send_message(message.from_user.id, 'Было приятно пообщаться! Хочешь узнать что я умею еще? Введи команду /back')

def choise_play(message):
    inp = message.text
    if inp.lower() == 'да':
      bot.send_message(message.from_user.id, play_start)
      bot.register_next_step_handler(message, play)
    else:
      bot.send_message(message.from_user.id, 'Было приятно поиграть! Хочешь узнать что я умею еще? Введи команду /back')

def choise_names(message):
    inp = message.text
    if inp.lower() == 'да':
      bot.send_message(message.from_user.id, game_start)
      bot.register_next_step_handler(message, names)
    else:
      bot.send_message(message.from_user.id, 'Было здорово придумать тебе имя! Хочешь узнать что я умею еще? Введи команду /back')
        
bot.infinity_polling()

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
